# **1. MinIO là gì?**

- MinIO là một hệ thống lưu trữ đối tượng (object storage) mã nguồn mở, tương tự như Amazon S3. Nó được thiết kế để lưu trữ và quản lý dữ liệu phi cấu trúc (ảnh, video, log, dữ liệu sensor, file backup, dataset lớn, v.v.).

🔑 Đặc điểm chính của MinIO:

- S3-compatible API: tương thích với Amazon S3 API, nên các ứng dụng viết cho S3 có thể chạy với MinIO gần như không cần chỉnh sửa.

- Triển khai linh hoạt: chạy được trên bare metal, VM, container, Kubernetes.

- Hiệu năng cao: được tối ưu cho big data workloads, AI/ML, analytics.

- Mã nguồn mở: bạn có thể dùng miễn phí hoặc mua enterprise support.

- Hỗ trợ scale-out: gom nhiều node thành 1 cluster lưu trữ dữ liệu petabyte.

- Bảo mật: hỗ trợ TLS, IAM, kiểm soát truy cập chi tiết.

📌 Ứng dụng thực tế:

- Làm data lake để lưu trữ dữ liệu thô từ nhiều nguồn (log, IoT, API, database).

- Lưu backup & archive.

- Làm storage backend cho Spark, Presto, Trino, Hive, dbt, Airflow...

- Phục vụ AI/ML training, khi cần đọc dữ liệu lớn từ file.

👉 Nói ngắn gọn: MinIO = S3 self-hosted (tự dựng) cho các dự án Data Engineering, Big Data, AI.

# **2. 🧊 Iceberg là gì?**

- Apache Iceberg = Table Format dành cho data lake.

- Nó không phải database, cũng không phải file format mới.

- Nhiệm vụ của Iceberg là biến một đống file Parquet/ORC/Avro trong data lake thành “bảng” giống như SQL table để query và quản lý dễ dàng.

🎯 Các điểm mạnh:
- Schema evolution: thay đổi schema (thêm/bỏ cột) mà không phá dữ liệu cũ.

- Time travel: quay ngược về snapshot trước đó (kiểu SELECT * FROM table VERSION AS OF '2025-08-01').

- ACID transaction: hỗ trợ update, delete, merge dữ liệu trong data lake.

- Partitioning thông minh: không cần nhớ partition path thủ công.

- Query engines hỗ trợ: Spark, Trino, Flink, Snowflake, Dremio, Athena…

- Nói ngắn gọn: Parquet = file; Iceberg = tổ chức thành bảng + metadata + snapshot.

# **3. Khi có minIO rồi thì cài iceberg trên nó à?**

- Không, cách mà nó thực sự hoạt động như ví dụ sau:
    + Ban đầu, data raw được lưu trong /raw-bucket/users dưới dạng các file Parquet:

In [ ]:
/raw-bucket/users/
   users1.parquet 
   users2.parquet 

    + Sau đó, khi chạy lệnh SQL dưới đây thì nó tạo ra 1 file metadata trong /iceberg-bucket/warehouse/users':

In [ ]:
CREATE TABLE local.db.users (
  id BIGINT,
  name STRING,
  age INT
) USING iceberg
LOCATION 's3a://iceberg-bucket/warehouse/users';


In [ ]:
/iceberg-bucket/warehouse/users/
   metadata/
      v1.metadata.json     # mô tả schema, partition, snapshot
      schema-1.avsc        # mô tả schema Avro
   data/                   # (chưa có file vì chưa insert)

    + Và bây giờ, các query engine (Ex: Spark, Trino) sẽ coi folder /iceberg-bucket/warehouse/users/ như là 1 bảng User có các hàng, cột đầy đủ
    => Tức là các query engine có thể truy vấn dữ liệu như bảng bình thường bằng SQL

In [ ]:
SELECT * 
FROM local.db.users
WHERE country = 'Vietnam';


    + Tuy nhiên, hãy để ý, trong /iceberg-bucket/warehouse/users/data hiện chưa có gì, do đó, cần phải INSERT dữ liệu từ /raw-bucket/ sang

In [ ]:
INSERT INTO local.db.users
SELECT * FROM parquet.`s3a://raw-bucket/users/users1.parquet`;

INSERT INTO local.db.users
SELECT * FROM parquet.`s3a://raw-bucket/users/users2.parquet`;


# **4. Snapshot là gì**

- Về cơ bản, khi 1 folder (ví dụ như /raw-bucket/users/) có bất cứ thứ gì thay đổi (thêm, sửa , xóa, merge) thì Iceberg không ghi đè file cũ mà tạo ra 1 snapshot mới => Snapshot = một bản chụp toàn bộ trạng thái bảng tại một thời điểm.
- Nó lưu:
    + Các data files (Parquet/ORC/Avro) nào đang thuộc về bảng.
    + Các delete files (nếu có).
    + Metadata tham chiếu đến chúng.

- Lợi ích:
    + Atomic & ACID: nhờ snapshot nên mọi thay đổi đều an toàn, hoặc commit hết hoặc không commit gì.
    + Rollback dễ dàng: nếu ETL sai, bạn chỉ cần ROLLBACK TO SNAPSHOT <id> là dữ liệu quay lại trạng thái trước đó.
    + Time travel: bạn có thể query dữ liệu "quay ngược thời gian":

In [ ]:
-- Query snapshot mới nhất (mặc định)
SELECT * FROM my_table;

-- Query snapshot theo ID cụ thể
SELECT * FROM my_table.snapshot_id = '123456789';

-- Query bảng tại thời điểm T trong quá khứ
SELECT * FROM my_table FOR TIMESTAMP AS OF '2025-08-01 10:00:00';


- Snapshot thì được lưu ở đâu?
    + Snapshot không phải là file riêng biệt kiểu snapshot-xxx.json.
    + Thông tin snapshot nằm trong metadata.json (v1.metadata.json, v2.metadata.json, …).
    + Trong file v2.metadata.json có một mảng snapshots chứa nhiều bản ghi, mỗi bản ghi là một snapshot.
- Ví dụ (rút gọn):

In [ ]:
{
  "format-version": 2,
  "table-uuid": "1234-5678",
  "snapshots": [
    {
      "snapshot-id": 111,
      "timestamp-ms": 1690000000000,
      "summary": { "operation": "append" },
      "manifest-list": "s3://bucket/db/table/metadata/snap-111.avro"
    },
    {
      "snapshot-id": 112,
      "timestamp-ms": 1690000500000,
      "summary": { "operation": "overwrite" },
      "manifest-list": "s3://bucket/db/table/metadata/snap-112.avro"
    }
  ],
  "current-snapshot-id": 112
}


- Như vậy:

    + v1.metadata.json thường chứa thông tin của 1 snapshot (ban đầu).

    + v2.metadata.json chứa thông tin của 2 snapshot (snapshot cũ + snapshot mới nhất).

    + current-snapshot-id chỉ ra snapshot đang active để đọc bảng.

**=> Cấu trúc folder Iceberg đầy đủ như sau:**

In [ ]:
my_table/
  metadata/
    v1.metadata.json
    v2.metadata.json
    ...
  data/
    <các file parquet>
  snap-*.avro
  manifest-*.avro


📌 Về File vật lý của snapshot

+ Mỗi snapshot tham chiếu đến một file gọi là manifest list (snap-xxx.avro).

+ Manifest list này lại tham chiếu đến nhiều manifest files (manifest-xxx.avro).

+ Các manifest file chứa danh sách các data files (Parquet/ORC) cụ thể.

⛓️ Chuỗi liên kết:
+ metadata.json → snap-xxx.avro → manifest-xxx.avro → *.parquet